In [1]:
# load the libraries

import urllib2, urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib2
import json
import time


In [2]:
risk_all = []
for i in range(1, 532): 
    path = '/Users/yangshenyang/Google\ Drive/course/web\ data/project/output/'+str(i)+'_risk.txt'
    risk = open(path)
    a= risk.read()
    risk_all.append(a)
# print risk_all

In [3]:
dis_all = []
for i in range(1, 532): # range(1,532)
    path = '/Users/yangshenyang/Google\ Drive/course/web\ data/project/output/'+str(i)+'_discussion.txt'
    dis = open(path)
    a= dis.read()
    dis_all.append(a)
# print dis_all

In [4]:
market_risk_all = []
for i in range(1, 532): # range(1,532)
    path = '/Users/yangshenyang/Google\ Drive/course/web\ data/project/output/'+str(i)+'_market_risk.txt'
    mr = open(path)
    mrr= mr.read()
    market_risk_all.append(mrr)
# print market_risk_all

In [7]:
# convert list of risk factor into list of score
scores_three_section = []
sec_names = ['risk', ' discussion', 'market risk']
for sec_id, section_data in enumerate([risk_all, dis_all, market_risk_all]):
    scores = []
    for i, data in enumerate(section_data):
        print "\n=== %s %i started ===" % (sec_names[sec_id], i)
        if data == 'N/A':
            print 'data N/A, skipped'
            scores.append('N/A')
            continue

        sentiment1 = []
        keyphrases1 = []
        total_keys1 = []
        avg_score1 = []
        final1 = []

        step = 4000 # has to be less than 5000
        num_step = int(len(data)/step)
        if num_step == 0:
            num_step = 1 # if length < 4000, just enforce 1 chunk

        print "data total length %s => %i chunks" % (len(data), num_step)
        for j in range(num_step):
            parts = data[j*step:(j+1)*step]
            parts = parts.encode('ascii',errors='ignore')
            parts = parts.replace("'", "").replace('"', '') # remove single and double quotes
            #Here we have broken down our text in to step character chunks
            
            # Azure portal URL.
            base_url = 'https://westus.api.cognitive.microsoft.com/'
            # The unique account key goes here.
            account_key = '9ad81715e6b2480f91711867a2a9e8c1'    # 10,000 calls a month free
            headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}
            input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
            #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

            # Detect key phrases.
            batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
            req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
            try:
                response = urllib2.urlopen(req)
                result = response.read()
                keyph = json.loads(result)
                for phrases in keyph['documents']:
                    keyphrases1.append(phrases['keyPhrases'])
                # print "phrases['keyPhrases']", phrases['keyPhrases']
            except Exception as e: 
                print "keyPhrases failed URL open, skip, error message: ", e
                
            # Detect sentiment.
            batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
            req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
            try:
                response = urllib2.urlopen(req)
                result = response.read()
                senti = json.loads(result)
                for sentiment_analysis in senti['documents']:
                    sentiment1.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list
                print "data %i, chunk %i/%i, sentiment %.3f" % (i, j, num_step, sentiment_analysis['score'])
            except Exception as e: 
                print "sentiment failed URL open, skip, error message: ", e
                
        # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
        # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

        for items in keyphrases1:
            for x in items:
                final1.append(x)

        unique_keys = list(set(final1)) #Using set functions, we clean the list to remove duplicates

        total_keys1.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

        avg_senti = np.mean(sentiment1) #Here we are calculate the average sentiment score for each of the chunks

        avg_score1.append(avg_senti) # Storing the average sentiment score in the average score list

        print "avg_senti calculated from %i/%i chunks: %f"% (len(sentiment1), num_step, avg_senti)
        scores.append(avg_score1)

    print '----final average scores for all section %s----' % sec_names[sec_id]
    print scores
    scores_three_section.append(scores)

print '----final average scores for all 3 section data----'
print scores_three_section


=== risk 0 started ===
data total length 47942 => 11 chunks
keyPhrases failed URL open, skip, error message:  HTTP Error 401: Access Denied
sentiment failed URL open, skip, error message:  HTTP Error 401: Access Denied
keyPhrases failed URL open, skip, error message:  HTTP Error 401: Access Denied
sentiment failed URL open, skip, error message:  HTTP Error 401: Access Denied
keyPhrases failed URL open, skip, error message:  HTTP Error 401: Access Denied
sentiment failed URL open, skip, error message:  HTTP Error 401: Access Denied
keyPhrases failed URL open, skip, error message:  HTTP Error 401: Access Denied
sentiment failed URL open, skip, error message:  HTTP Error 401: Access Denied
keyPhrases failed URL open, skip, error message:  HTTP Error 401: Access Denied


KeyboardInterrupt: 

In [ ]:
from textblob import TextBlob
blob = TextBlob(market_risk_all[1])
nlob

In [ ]:
dis_all = []
for i in range(1, 532): # range(1,532)
    path = '/Users/yangshenyang/Google\ Drive/course/web\ data/project/output/'+str(i)+'_discussion.txt'
    dis = open(path)
    a= dis.read()
    dis_all.append(a)
print dis_all

In [ ]:
df = pd.read_csv('test.txt',header=None) # Here we are reading the 10K submission links

links = list(df[0])
links = ['https://www.sec.gov/Archives/edgar/data/1050797/000105079717000013/colm-20161231x10k.htm#s37E44109873A5A479DBABAB66B750F24']
print links

In [ ]:
# get the list of item 1A: risk factors

risk1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    print html
#     index = re.search('item 1a', html,flags=re.IGNORECASE) # searches the submission file for the exact mention of the term '1A'
#     print '1st index', index.start()
#     html = html[index.start()+3:] #cuts the html after the first 1A occurence, which is after the table of contents
#     index = re.search('item 1a', html,flags=re.IGNORECASE) # finds 1A in the html after the table of contents
#     print '2nd index', index.start()
#     html = html[index.start():] # cuts off the previous html part
#     index = re.search('item 1b', html,flags=re.IGNORECASE) # finds the exact occurence of 1B
#     print "3rd index:", index
#     
#     risk1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

risk2 = []

for i in range(len(risk1)):
    soup = BeautifulSoup(risk1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    risk2.append(text.encode('utf-8'))

In [ ]:
risk2

In [ ]:
# get the list of item 7: Management Discussion

md1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    index = re.search('Item 7.', html) # searches the submission file for the exact mention of the term 'Item 7.'
    html = html[index.start()+3:] #cuts the html after the first Item 7. occurence, which is after the table of contents
    index = re.search('ITEM 7.', html) # finds ITEM 7. in the html after the table of contents
    html = html[index.start():] # cuts off the previous html part
    index = re.search('ITEM 7A.', html) # finds the exact occurence of ITEM 7A.
    md1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

md2 = []

for i in range(len(md1)):
    soup = BeautifulSoup(md1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    md2.append(text.encode('utf-8'))

md2

In [ ]:
# get the list of item 7A: Market Risk

market1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    index = re.search('Item 7A.', html) # searches the submission file for the exact mention of the term 'Item 7.'
    html = html[index.start()+3:] #cuts the html after the first Item 7. occurence, which is after the table of contents
    index = re.search('ITEM 7A.', html) # finds ITEM 7. in the html after the table of contents
    html = html[index.start():] # cuts off the previous html part
    index = re.search('ITEM 8.', html) # finds the exact occurence of ITEM 7A.
    market1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

market2 = []

for i in range(len(market1)):
    soup = BeautifulSoup(market1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    market2.append(text.encode('utf-8'))

In [ ]:
avg_score1

In [ ]:
# convert list of Management Discussion into list of score

for i in range(len(md2)):
    
    sentiment2 = []
    keyphrases2 = []
    total_keys2 = []
    avg_score2 = []
    
    final2 = []
    text = md2[i]
    for x in range(0, len(text), 2000):
        if len(text)-x > 2000:
            parts = text[x:x+2000]
        else:
            parts = text[x:]

        #Here we have broken down our text in to 2000 character chunks

        # Azure portal URL.
        base_url = 'https://westus.api.cognitive.microsoft.com/'
        # The unique account key goes here.
        account_key = '9ad81715e6b2480f91711867a2a9e8c1'

        headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

        input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
        #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

        # Detect key phrases.
        batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
        req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        keyph = json.loads(result)

        for phrases in keyph['documents']:
            keyphrases2.append(phrases['keyPhrases'])

        # Detect sentiment.
        batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
        req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        senti = json.loads(result)

        for sentiment_analysis in senti['documents']:
            sentiment2.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list

    # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
    # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

    for items in keyphrases2:
        for x in items:
            final2.append(x)

    unique_keys = list(set(final2)) #Using set functions, we clean the list to remove duplicates

    total_keys2.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

    avg_senti = np.mean(sentiment2) #Here we are calculate the average sentiment score for each of the chunks

    avg_score2.append(avg_senti) # Storing the average sentiment score in the average score list

In [ ]:
# convert list of Management Discussion into list of score

for i in range(len(market2)):
    
    sentiment3 = []
    keyphrases3 = []
    total_keys3 = []
    avg_score3 = []
    
    final3 = []
    text = market2[i]
    for x in range(0, len(text), 2000):
        if len(text)-x > 2000:
            parts = text[x:x+2000]
        else:
            parts = text[x:]

        #Here we have broken down our text in to 2000 character chunks

        # Azure portal URL.
        base_url = 'https://westus.api.cognitive.microsoft.com/'
        # The unique account key goes here.
        account_key = '01f5b19c9bd9425ab5c17dfaf2a497ed'

        headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

        input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
        #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

#         # Detect key phrases.
        batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
#         req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
#         response = urllib2.urlopen(req)
#         result = response.read()
#         keyph = json.loads(result)

#         for phrases in keyph['documents']:
#             keyphrases3.append(phrases['keyPhrases'])

#         # Detect sentiment.
        batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
#         req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
#         response = urllib2.urlopen(req)
#         result = response.read()
#         senti = json.loads(result)

#         for sentiment_analysis in senti['documents']:
#             sentiment3.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list

        import requests
        tufani={
             "documents": [
                 {
                     "id": "1",
                     "text": parts
                 },
             ]
         }
        r=requests.post(batch_sentiment_url,data=tufani,headers =headers)
        print(r.status_code, r.reason)


        
    # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
    # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

    for items in keyphrases2:
        for x in items:
            final2.append(x)

    unique_keys = list(set(final3)) #Using set functions, we clean the list to remove duplicates

    total_keys3.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

    avg_senti = np.mean(sentiment3) #Here we are calculate the average sentiment score for each of the chunks

    avg_score3.append(avg_senti) # Storing the average sentiment score in the average score list

In [ ]:
avg_score3